**filtering out doublets using DoubletFinder**

In [1]:
#stting working directory
setwd("/home/mmokhtari/INF_analysis")

In [2]:
#loading libraries
library(Seurat)
library(tidyverse)
library(ggplot2)
library(DoubletFinder)

Attaching SeuratObject

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────── tidyverse 1.3.2.9000 ──
✔ dplyr     1.0.10     ✔ readr     2.1.3 
✔ forcats   0.5.2      ✔ stringr   1.4.1 
✔ ggplot2   3.4.0      ✔ tibble    3.1.8 
✔ lubridate 1.9.0      ✔ tidyr     1.2.1 
✔ purrr     0.3.5      
── Conflicts ──────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [3]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /home/mmokhtari/.conda/envs/r_env/lib/libopenblasp-r0.3.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] DoubletFinder_2.0.3  lubridate_1.9.0      timechange_0.1.1    
 [4] forcats_0.5.2        stringr_1.4.1        dplyr_1.0.10        
 [7] purrr_0.3.5          readr_2.1.3          tidyr_1.2.1         
[10] tibble_3.1.8         ggplot2_3.4.0        tidyverse_1.3.2.9000
[13] SeuratObject_4.1.3   Seura

In [4]:
#loading the donor_ids
donor_id <- read.table("/home/mmokhtari/INF_analysis/donor_ids.tsv",header = TRUE)
row.names(donor_id) <-donor_id$cell
head(donor_id)

,cell,donor_id,prob_max,prob_doublet,n_vars,best_singlet,best_doublet,doublet_logLikRatio
,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>
AAACCCAAGACTGGGT-1,AAACCCAAGACTGGGT-1,donor1,1.00e+00,8.99e-40,215,donor1,"donor0,donor1",-89.708
AAACCCAAGAGCTGCA-1,AAACCCAAGAGCTGCA-1,doublet,3.78e-07,1.00e+00,70,donor1,"donor0,donor1",14.988
AAACCCAAGGATATGT-1,AAACCCAAGGATATGT-1,donor1,1.00e+00,3.07e-103,995,donor1,"donor0,donor1",-235.846
AAACCCAAGGATGGCT-1,AAACCCAAGGATGGCT-1,unassigned,6.29e-01,2.91e-01,2,donor0,"donor0,donor1",-0.573
AAACCCAAGGGATCGT-1,AAACCCAAGGGATCGT-1,donor0,1.00e+00,7.48e-16,161,donor0,"donor0,donor1",-34.630
AAACCCAAGGTCTTTG-1,AAACCCAAGGTCTTTG-1,doublet,6.83e-09,1.00e+00,120,donor1,"donor0,donor1",19.002


In [5]:
genetic_doublets <- donor_id[which(donor_id$donor_id== "doublet"),1]

In [6]:
#loading count matrices
data_raw <- Read10X_h5("/home/mmokhtari//INF_analysis/raw_feature_bc_matrix.h5", use.names = TRUE, unique.features = TRUE)

In [7]:
#ckecking the data loaded
class(data_raw)
dim(data_raw)
head(data_raw)

[1] "dgCMatrix"
attr(,"package")
[1] "Matrix"

[1]   20639 1395467

   [[ suppressing 34 column names ‘AAACCCAAGAAACACT-1’, ‘AAACCCAAGAAACCCA-1’, ‘AAACCCAAGAAACTAC-1’ ... ]]



6 x 1395467 sparse Matrix of class "dgCMatrix"
                                                                            
TNFRSF4  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
TNFRSF18 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
ATAD3B   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
PRDM16   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
ACAP3    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
PUSL1    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
               
TNFRSF4  ......
TNFRSF18 ......
ATAD3B   ......
PRDM16   ......
ACAP3    ......
PUSL1    ......

 .....suppressing 1395433 columns in show(); maybe adjust 'options(max.print= *, width = *)'
 ..............................

In [8]:
#creating the Seurat object
data <- CreateSeuratObject(counts = data_raw)
data

An object of class Seurat 
20639 features across 1395467 samples within 1 assay 
Active assay: RNA (20639 features, 0 variable features)

In [9]:
Idents(data)<- row.names(data@meta.data)
#Idents(data)==genetic_doublets

In [10]:
#cells.use <- data[["nCount_RNA", drop = TRUE]] <= 4000 &
 # data[["nCount_RNA", drop = TRUE]] >= 200 &
  #data[["nFeature_RNA", drop = TRUE]] <= 5000 &
  #data[["nFeature_RNA", drop = TRUE]] >= 100

In [11]:
# Remove filtered cells from the query
#data <- data[, cells.use]

In [12]:
head(data$orig.ident)

AAACCCAAGAAACACT-1 AAACCCAAGAAACCCA-1 AAACCCAAGAAACTAC-1 AAACCCAAGAAAGACA-1 
     SeuratProject      SeuratProject      SeuratProject      SeuratProject 
AAACCCAAGAAAGCGA-1 AAACCCAAGAAATGGG-1 
     SeuratProject      SeuratProject 
Levels: SeuratProject

In [13]:
any(row.names(data@meta.data)=="AAACCCAAGGATGGCT-1")

[1] TRUE

In [16]:
#QC and filtering
data$mitoPercent <-PercentageFeatureSet(data,pattern = "^MT")
data_filtered <- subset(data, subset = nCount_RNA > 800 &
         nFeature_RNA > 500 &
         mitoPercent < 10)

In [ ]:
data_filtered@meta.data$donor_id <- donor_id[row.names(data_filtered@meta.data),2]

In [ ]:
data_filtered<- subset(x = data_filtered, subset = donor_id == c("doublet","unassigned"), invert = TRUE)

In [ ]:
data_filtered@meta.data

In [ ]:
Idents(data_filtered) <- data_filtered@meta.data$orig.ident


In [ ]:
dim(na.omit(data@meta.data))

In [ ]:
any(data_filtered$orig.ident==donor_id$cell)

In [ ]:
dim(data_filtered)

In [ ]:
# pre-process standard workflow
data_filtered <- NormalizeData(object = data_filtered)
data_filtered <- FindVariableFeatures(object = data_filtered)
data_filtered <- ScaleData(object = data_filtered)
data_filtered <- RunPCA(object = data_filtered)
ElbowPlot(data_filtered)

In [ ]:
data_filtered <- FindNeighbors(object = data_filtered, dims = 1:13)
data_filtered <- FindClusters(object = data_filtered)
data_filtered <- RunUMAP(object = data_filtered, dims = 1:20)

In [ ]:
## pK Identification (no ground-truth) ---------------------------------------------------------------------------------------
sweep.res.list_data <- paramSweep_v3(data_filtered, PCs = 1:13, sct = FALSE)
sweep.stats_data <- summarizeSweep(sweep.res.list_data, GT = FALSE)
bcmvn_data <- find.pK(sweep.stats_data)


In [ ]:
ggplot(bcmvn_data, aes(pK, BCmetric, group = 1)) +
  geom_point() +
  geom_line()


In [ ]:
pK <- bcmvn_data %>% # select the pK that corresponds to max bcmvn to optimize doublet detection
  filter(BCmetric == max(BCmetric)) %>%
  select(pK) 
pK <- as.numeric(as.character(pK[[1]]))

In [ ]:
pK

In [ ]:
## Homotypic Doublet Proportion Estimate -------------------------------------------------------------------------------------
annotations <- data_filtered@meta.data$seurat_clusters
homotypic.prop <- modelHomotypic(annotations)           ## ex: annotations <- seu_kidney@meta.data$ClusteringResults
nExp_poi <- round(0.09*nrow(data_filtered@meta.data))  ## Assuming 9% doublet formation rate - tailor for your dataset
nExp_poi.adj <- round(nExp_poi*(1-homotypic.prop))

In [ ]:
nExp_poi
nExp_poi.adj

In [ ]:
# run doubletFinder 
data_filtered <- doubletFinder_v3(data_filtered, 
                                     PCs = 1:13, 
                                     pN = 0.25, 
                                     pK = pK, 
                                     nExp = nExp_poi.adj,
                                     reuse.pANN = FALSE, sct = FALSE)

In [ ]:
colnames(data_filtered@meta.data)

In [ ]:
# visualize doublets
DimPlot(data_filtered, reduction = 'umap', group.by = "DF.classifications_0.25_0.005_1405")

In [ ]:
write.table(data_filtered@meta.data,"doublets_1.tsv", sep="\t")

In [ ]:
colnames(data_filtered@meta.data)

In [ ]:
data_filtered

In [4]:
doublets <- read.table("doublets.tsv")
#doublets_1 <- read.table("doublets_1.tsv")


In [5]:
doublets

,orig.ident,nCount_RNA,nFeature_RNA,mitoPercent,donor_id,RNA_snn_res.0.8,seurat_clusters,pANN_0.25_0.005_1405,DF.classifications_0.25_0.005_1405
,<chr>,<int>,<int>,<dbl>,<chr>,<int>,<int>,<dbl>,<chr>
AAACCCAAGACTGGGT-1,SeuratProject,3960,1051,3.0555556,donor1,6,6,0.076923077,Singlet
AAACCCAAGAGCTGCA-1,SeuratProject,576,400,3.1250000,doublet,1,1,0.085470085,Singlet
AAACCCAAGGATATGT-1,SeuratProject,7565,2930,1.5598149,donor1,3,3,0.324786325,Singlet
AAACCCAAGGGATCGT-1,SeuratProject,1383,887,1.3015184,donor0,10,10,0.188034188,Singlet
AAACCCAAGTAGTCCT-1,SeuratProject,708,468,2.9661017,donor1,1,1,0.128205128,Singlet
AAACCCACAAGATGGC-1,SeuratProject,639,486,2.6604069,donor1,1,1,0.111111111,Singlet
AAACCCACAATGTCAC-1,SeuratProject,3370,1512,2.2551929,donor0,9,9,0.247863248,Singlet
AAACCCACAGATCCAT-1,SeuratProject,4135,1974,2.9020556,donor0,9,9,0.256410256,Singlet
AAACCCAGTACGGCAA-1,SeuratProject,827,561,4.2321644,doublet,1,1,0.264957265,Singlet


In [7]:
length((doublets$DF.classifications_0.25_0.005_1405))

[1] 17604

In [10]:
doublets$classifications_0.25_0.005_1405=="Singlet"

logical(0)

In [12]:
doublets$DF.classifications_0.25_0.005_1405=="Singlet"

[1]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
   [13]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
   [25]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
   [37]  TRUE  TRUE FALSE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
   [49] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
   [61]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
   [73] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
   [85]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
   [97] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [109]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [121] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
  [133]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [145]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE
  [157]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [169]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [181]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [193]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
  [205] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
  [217]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [229]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
  [241]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE
  [253] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [265]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
  [277]  TRUE FALSE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [289]  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE
  [301]  TRUE  TRUE  TRUE  TRUE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [313]  TRUE  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [325]  TRUE  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
  [337]  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [349] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [361]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE
  [373]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [385]  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
  [397]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [409] FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [421]  TRUE  TRUE  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
  [433]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [445]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [457] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [469]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE
  [481]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
  [493]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [505]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
  [517]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [529]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [541]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [553]  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [565]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [577]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [589]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [6

In [20]:
min(doublets$pANN_0.25_0.005_1405[doublets$DF.classifications_0.25_0.005_1405=="Doublet",])

ERROR: Error in doublets$pANN_0.25_0.005_1405[doublets$DF.classifications_0.25_0.005_1405 == : incorrect number of dimensions


In [24]:
dd <- doublets[doublets$DF.classifications_0.25_0.005_1405=="Doublet",]

In [36]:
length(which(dd$pANN_0.25_0.005_1405<0.40))

[1] 377

In [42]:
min(dd$pANN_0.25_0.005_1405)

[1] 0.3589744

In [37]:
dim(doublets[doublets$DF.classifications_0.25_0.005_1405=="Doublet",])

[1] 1405    9

In [41]:
length(which(doublets$DF.classifications_0.25_0.005_1405=="Doublet"))/length(which(doublets$DF.classifications_0.25_0.005_1405=="Singlet"))

[1] 0.08673375

In [3]:
read.table("doublets_1.tsv")

Warning message in file(file, "rt"):
“cannot open file 'doublets_1.tsv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection
